In [ ]:
using InMemoryDatasets
import InMemoryDatasets as IMD
using BenchmarkTools
using TimerOutputs


In [122]:
include("../src/main.jl")

cartesianjoin (generic function with 1 method)

# Data

In [ ]:
@show dsl = Dataset(xid = [111,222,333,444,555], 
                    x1 = [1,2,1,4,5], 
                    x2 = [-1.2,-3,2.1,-3.5,2.8],
                    x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")],
                    x4 = ["abcd","efgh","ijkl","mnop","qrst"]);

@show dsr = Dataset(yid = [111,111,222,444,333],
                    y1 = [3,3,3,3,3],
                    y2 = [0,0, -3,1,2],
                    y3 = [Date("2019-10-01"),Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")],
                    y4 = ["abc","abcd","efg","mnop","qrst"]);

dsl_big = Dataset(rand(10^3, 5), :auto);
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big = Dataset(rand(10^3, 5), :auto);
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big);

dsl_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big2);

# Working

In [ ]:
innerjoin(dsl,dsr,on = 1)  

## on syntax

In [ ]:
on = [1 => 1, 3 => 3]
on1 = ["xid" => "yid","x1"=>"y1"]
on2 = [(:xid => :yid) => isequal, (:x1 => :y1) => isless]
on3  = [:xid => :yid, :x1 => :y1 => isless]
#on4  = [:xid => :yid, :x1 => :y1]

In [ ]:
# 两边都有同名列，on = :id 用的
(typeof(on1) <: AbstractVector{<:Union{AbstractString, Symbol}})

In [ ]:
(typeof(on2) <: AbstractVector{<:Pair{<:Pair{<:IMD.ColumnIndex,<:IMD.ColumnIndex}, <:Function}})

In [ ]:
typeof(on3)

In [ ]:
on3  = [:xid => :yid, :x1 => :y1 => isless]
if (typeof(on3) <: AbstractVector{<:Pair{<:IMD.ColumnIndex, <:Any}})

  dsr_cols = Symbol[]
  equalon_dsr_cols = Symbol[]
  conditions = Function[]

  for element in map(x -> x.second, on3)
    if typeof(element) <: Pair
      !(element.second isa Function) && throw(ArgumentError("Need Function"))
      push!(dsr_cols, element.first)
      push!(conditions, element.second)
    elseif typeof(element) <: IMD.ColumnIndex
      push!(dsr_cols, element)
      push!(equalon_dsr_cols, element)
      push!(conditions, isequal)
    else
      
      throw(ArgumentError("error `on`"))
    end
  end

  onleft = IMD.multiple_getindex(IMD.index(dsl), map(x -> x.first, on))
  onright = IMD.multiple_getindex(IMD.index(dsr), dsr_cols)
  onright_equal = IMD.multiple_getindex(IMD.index(dsr), equalon_dsr_cols)

end

# Test

In [ ]:
function fun1(x,y) 
  x <= y
end

In [123]:
newds = cartesianjoin(dsl,dsr,on = [:xid=>:yid, :x2=>:y2=>fun1],multiple_match=true);
nds

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            627ms / 100.0%           41.1MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    505ms   80.6%   505ms   28.6MiB   69.6%  28.6MiB
   2-2 left row _res        1    329ms   52.4%   329ms   18.0MiB   43.7%  18.0MiB
     2.2 fill _res          5    328ms   52.4%  65.6ms   17.9MiB   43.7%  3.59MiB
     2.1 init _res          5    288μs    0.0%  57.5μs      784B    0.0%     157B
     2.3 push _res          5   10.8μs    0.0%  2.15μs     80.0B    0.0%    16.0B
   2-5 newds dsr            1    176ms   28.1%   176ms   10.6MiB   25.9%  10.6MiB
     5.2 fill _

4×9 Dataset
 Row │ xid       x1        x2        x3          x4        y1        y2        ⋯
     │ identity  identity  identity  identity    identity  identity  identity  ⋯
     │ Int64?    Int64?    Float64?  Date?       String?   Int64?    Int64?    ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  2019-10-03  abcd             3         0  ⋯
   2 │      111         1      -1.2  2019-10-03  abcd             3         0
   3 │      222         2      -3.0  2019-09-30  efgh             3        -3
   4 │      444         4      -3.5  2019-10-03  mnop             3         1
                                                               2 columns omitted

In [124]:
cartesianjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2=>fun1, :x3=>:y3=>isless],
check=false);

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            396ms / 100.0%           46.4MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    217ms   54.8%   217ms   39.5MiB   85.0%  39.5MiB
   2-1 left row count       1    125ms   31.6%   125ms    117KiB    0.2%   117KiB
   2-5 newds dsr            1   55.9ms   14.1%  55.9ms   12.2MiB   26.3%  12.2MiB
     5.2 fill _res          4   54.8ms   13.8%  13.7ms   3.55MiB    7.6%   908KiB
     5.1 init _res          4    773μs    0.2%   193μs   8.64MiB   18.6%  2.16MiB
   2-4 findall dsr idx      1   31.9ms    8.0%  31.9ms   16.4MiB   35.2%  16.4MiB
     4.1 findal

In [127]:
nds = cartesianjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2, :x3=>:y3],check=false);
nrow(nds)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            261ms / 100.0%           13.5MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    166ms   63.4%   166ms   12.1MiB   89.9%  12.1MiB
   2-1 left row count       1    140ms   53.5%   140ms    117KiB    0.9%   117KiB
   2-4 findall dsr idx      1   24.8ms    9.5%  24.8ms   11.9MiB   88.7%  11.9MiB
     4.1 findall func       1   24.7ms    9.5%  24.7ms   11.9MiB   88.6%  11.9MiB
   2-2 left row _res        1    745μs    0.3%   745μs   25.0KiB    0.2%  25.0KiB
     2.2 fill _res          5    570μs    0.2%   114μs   18.2KiB    0.1%  3.65KiB
     2.1 init _

86

In [129]:
@btime nds = innerjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2, :x3=>:y3],check=false);
nrow(nds)

  1.980 ms (1686 allocations: 682.02 KiB)


86

## multiple_match

In [120]:
newds, dsl_count, total_length = 
  cartesianjoin(dsl,dsr,on = [:xid=>:yid=>isless],multiple_match=true)

(6×11 Dataset
 Row │ xid       x1        x2        x3          x4        yid       y1        ⋯
     │ identity  identity  identity  identity    identity  identity  identity  ⋯
     │ Int64?    Int64?    Float64?  Date?       String?   Int64?    Int64?    ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  2019-10-03  abcd           222         3  ⋯
   2 │      111         1      -1.2  2019-10-03  abcd           444         3
   3 │      111         1      -1.2  2019-10-03  abcd           333         3
   4 │      222         2      -3.0  2019-09-30  efgh           444         3
   5 │      222         2      -3.0  2019-09-30  efgh           333         3  ⋯
   6 │      333         1       2.1  2019-10-04  ijkl           444         3
                                                               4 columns omitted, Int32[3, 2, 1, 0, 0], 6)

In [118]:
newds[!,[1,2,3,5,6,7,9,10,11]]


6×9 SubDataset
 Row │ xid       x1        x2        x4        yid       y1        y3          ⋯
     │ identity  identity  identity  identity  identity  identity  identity    ⋯
     │ Int64?    Int64?    Float64?  String?   Int64?    Int64?    Date?       ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  abcd           222         3  2019-09-30  ⋯
   2 │      111         1      -1.2  abcd           444         3  2019-10-05
   3 │      111         1      -1.2  abcd           333         3  2019-10-05
   4 │      222         2      -3.0  efgh           444         3  2019-10-05
   5 │      222         2      -3.0  efgh           333         3  2019-10-05  ⋯
   6 │      333         1       2.1  ijkl           444         3  2019-10-05
                                                               2 columns omitted

In [96]:
total_length

4

In [103]:
_create_multiple_match_col_cartesian(dsl_count, nothing, total_length)

4-element Vector{Union{Missing, Bool}}:
  true
  true
 false
 false

_create_multiple_match_col_cartesian (generic function with 1 method)

## obsid

In [ ]:
if obs_id[1]
  obs_id_name1 = Symbol(obs_id_name, "_left")
  obs_id_left = allocatecol(nrow(dsl) < typemax(Int32) ? Int32 : Int64, total_length)
  ## 和填充dsl用的同一个函数，研究一下
  _fill_oncols_left_table_inner!(obs_id_left, 1:nrow(dsl), ranges, new_ends, total_length; inbits = inbits, en2 = revised_ends, threads = threads)
  insertcols!(newds, ncol(newds)+1, obs_id_name1 => obs_id_left, unsupported_copy_cols = false, makeunique = makeunique)
end

if obs_id[2]
  obs_id_name2 = Symbol(obs_id_name, "_right")
  obs_id_right = allocatecol(T, total_length)
  _fill_right_cols_table_inner!(obs_id_right, idx, ranges, new_ends, total_length; inbits = inbits, en2 = revised_ends, threads = threads)
  insertcols!(newds, ncol(newds)+1, obs_id_name2 => obs_id_right, unsupported_copy_cols = false, makeunique = makeunique)
end

## mapformats
获取格式然后做值对比的时候套用format

In [ ]:
if mapformats[1]
  _fl = _date_value∘getformat(dsl, onleft[1])
end
if mapformats[2]
  _fr = _date_value∘getformat(dsr, onright[1])
end

if lmf
  _fl = getformat(dsl, l_idx)
else
  _fl = identity
end
if rmf
  _fr = getformat(dsr, r_idx)
else
  _fr = identity
end

if _fr == identity
  var_r_cpy = var_r
else
  var_r_cpy = map(_fr, var_r)
end

unwrap(x) = x
_flx = _fl(DataAPI.unwrap(x[i]))::T1